In [1]:
!pip install \
    langchain-pinecone \
    langchain-openai \
    langchain-text-splitters \
    langchain

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

with open("/home/choice/Desktop/whatsapp-chatbot-choice/data/Insurance.txt", 'r') as filename:
    markdown_document = filename.read()

headers_to_split_on = [
    ("\n\n")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)


In [4]:
from langchain_openai import OpenAIEmbeddings
import os

model_name = "text-embedding-3-small"  
embeddings = OpenAIEmbeddings(  
    model=model_name,  
    openai_api_key=os.environ.get("OPENAI_API_KEY")  
)  


In [5]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "docs-rag-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 


/home/choice/.local/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
import time
from langchain_pinecone import PineconeVectorStore

namespace = "wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=md_header_splits,
    index_name=index_name,
    embedding=embeddings, 
    namespace=namespace 
)

time.sleep(1)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 15 Jul 2024 11:54:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '888', 'x-pinecone-request-id': '6885482014277865029', 'x-envoy-upstream-service-time': '45', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 46408 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}
